#### This notebook provides the size of baseline models parameters.
#### N.B.: Numerous models exist in baseline methods. The models that exist in this notebook are the ones that were used to be compared with TPFL model. For FLIS baseline for instance, pretrained models are used as well that are not included in here.

#### IFCA baseline

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np

IMAGE_SIZE = 28

class ClientModel:
    def __init__(self, seed, lr, num_classes):
        self.num_classes = num_classes
        self.seed = seed
        self.lr = lr
        self.optimizer = tf.train.GradientDescentOptimizer(learning_rate=self.lr)

    def create_model(self):
        """Model function for CNN."""
        features = tf.placeholder(tf.float32, shape=[None, IMAGE_SIZE, IMAGE_SIZE, 1], name='features')
        labels = tf.placeholder(tf.int64, shape=[None], name='labels')

        conv1 = tf.keras.layers.Conv2D(filters=32, kernel_size=[5, 5], padding="same", activation='relu')(features)
        pool1 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2)(conv1)
        conv2 = tf.keras.layers.Conv2D(filters=64, kernel_size=[5, 5], padding="same", activation='relu')(pool1)
        pool2 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2)(conv2)
        pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
        dense = tf.keras.layers.Dense(units=2048, activation='relu')(pool2_flat)
        logits = tf.keras.layers.Dense(units=self.num_classes)(dense)

        predictions = {
            "classes": tf.argmax(input=logits, axis=1),
            "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
        }

        loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

        eval_metric_ops = tf.count_nonzero(tf.equal(labels, predictions["classes"]))
        self.trainable_variables = tf.trainable_variables()

        train_op = self.optimizer.minimize(
            loss=loss,
            var_list=self.trainable_variables
        )

        return features, labels, train_op, eval_metric_ops, loss

    def process_x(self, raw_x_batch):
        return np.array(raw_x_batch)

    def process_y(self, raw_y_batch):
        return np.array(raw_y_batch)
import numpy as np

def calculate_model_size(trainable_variables):
    total_params = 0

    for var in trainable_variables:
        var_shape = var.shape.as_list()
        var_size = np.prod(var_shape)
        total_params += var_size

    # Assuming each parameter is a float32 (4 bytes)
    total_size_bytes = total_params * 4  # 4 bytes per parameter (float32)
    total_size_megabytes = total_size_bytes / (1024 * 1024)  # bytes to megabytes

    return total_params, total_size_megabytes

model1 = ClientModel(seed=123, lr=0.01, num_classes=10)
model2 = ClientModel(seed=123, lr=0.01, num_classes=62)
features, labels, train_op, eval_metric_ops, loss = model1.create_model()
features, labels, train_op, eval_metric_ops, loss = model2.create_model()

print("10 classes")
total_params, total_size_megabytes = calculate_model_size(model1.trainable_variables)
print(f"Total number of parameters: {total_params}")
print(f"Total model size: {total_size_megabytes:.2f} MB")

print("62 classes")
total_params, total_size_megabytes = calculate_model_size(model2.trainable_variables)
print(f"Total number of parameters: {total_params}")
print(f"Total model size: {total_size_megabytes:.2f} MB")

Instructions for updating:
non-resource variables are not supported in the long term


10 classes
Total number of parameters: 6497162
Total model size: 24.78 MB
62 classes
Total number of parameters: 13100872
Total model size: 49.98 MB


##### FLIS CNN baseline

In [ ]:
import torch.nn as nn
import numpy as np

class SimpleCNNMNIST(nn.Module):
    def __init__(self, input_dim, hidden_dims, output_dim=10):
        super(SimpleCNNMNIST, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(input_dim, hidden_dims[0])
        self.fc2 = nn.Linear(hidden_dims[0], hidden_dims[1])
        self.fc3 = nn.Linear(hidden_dims[1], output_dim)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 4 * 4)

        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

class SimpleCNNMNIST2(nn.Module):
    def __init__(self, input_dim, hidden_dims, output_dim=62):
        super(SimpleCNNMNIST2, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(input_dim, hidden_dims[0])
        self.fc2 = nn.Linear(hidden_dims[0], hidden_dims[1])
        self.fc3 = nn.Linear(hidden_dims[1], output_dim)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 4 * 4)

        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

def calculate_model_size(model):
    total_params = 0
    for name, param in model.named_parameters():
        param_shape = param.shape
        var_size = np.prod(param_shape)
        total_params += var_size

    total_size_bytes = total_params * 4  # 4 bytes per parameter (float32)
    total_size_megabytes = total_size_bytes / (1024 * 1024)  # bytes to megabytes

    return total_params, total_size_megabytes

input_dim = 28 * 28
hidden_dims = [120, 84]
model1 = SimpleCNNMNIST(input_dim=input_dim, hidden_dims=hidden_dims)
model2 = SimpleCNNMNIST2(input_dim=input_dim, hidden_dims=hidden_dims)

# Calculate the size
total_params, total_size_megabytes = calculate_model_size(model1)
print(f"Total number of parameters: {total_params}")
print(f"Total model size: {total_size_megabytes:.2f} MB")

# Calculate the size
total_params, total_size_megabytes = calculate_model_size(model2)
print(f"Total number of parameters: {total_params}")
print(f"Total model size: {total_size_megabytes:.2f} MB")

Total number of parameters: 107786
Total model size: 0.41 MB
Total number of parameters: 112206
Total model size: 0.43 MB


#### FedAvg and FedProx baseline

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers, models, optimizers
def create_cnn_model():
    return models.Sequential([
        layers.Conv2D(20, kernel_size=(5, 5), activation='relu', input_shape=(28, 28, 1)),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(50, kernel_size=(5, 5), activation='relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(500, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])

def create_cnn_model2():
    return models.Sequential([
        layers.Conv2D(20, kernel_size=(5, 5), activation='relu', input_shape=(28, 28, 1)),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(50, kernel_size=(5, 5), activation='relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(500, activation='relu'),
        layers.Dense(62, activation='softmax')
    ])

def calculate_model_size(model):
    total_params = 0
    for layer in model.layers:
        for var in layer.trainable_variables:
            var_shape = var.shape.as_list()
            var_size = np.prod(var_shape)
            total_params += var_size

    total_size_bytes = total_params * 4  # 4 bytes per parameter (float32)
    total_size_megabytes = total_size_bytes / (1024 * 1024)  # bytes to megabytes

    return total_params, total_size_megabytes

model1 = create_cnn_model()
model2 = create_cnn_model2()

print("10 classes")
total_params, total_size_megabytes = calculate_model_size(model1)
print(f"Total number of parameters: {total_params}")
print(f"Total model size: {total_size_megabytes:.2f} MB")

print("62 classes")
total_params, total_size_megabytes = calculate_model_size(model2)
print(f"Total number of parameters: {total_params}")
print(f"Total model size: {total_size_megabytes:.2f} MB")

10 classes
Total number of parameters: 431080
Total model size: 1.64 MB
62 classes
Total number of parameters: 457132
Total model size: 1.74 MB


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
